In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, InputLayer, Input, BatchNormalization
import optuna
import openpyxl
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping
from keras.losses import SparseCategoricalCrossentropy
from keras.losses import MeanAbsoluteError
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
! pip install scikit-learn

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB 1.3 MB/s eta 0:00:09
   ---------------------------------------- 0.0/11.0 MB 393.8 kB/s eta 0:00:28
   ---------------------------------------- 0.1/11.0 MB 930.9 kB/s eta 0:00:12
    --------------------------------------- 0.2/11.0 MB 1.2 MB/s eta 0:00:10
    --------------------------------------- 0.2/11.0 MB 1.3 MB/s eta 0:00:09
   - -------------------------------------- 0.4/11.0 MB 1.4 MB/s eta 0:00:08
   - -------------------------------------- 0.4/11.0 MB 1.3 MB/s eta 0:00:08
   -- ------------------------------------- 0.6/11.0 MB 1.8 MB/s eta 0:00:06
   --- ------------------------------------ 0.8/11.0 MB 2.0 MB/s eta 0:00:06
   ---- ----------------------------------- 1.2/11.0 MB 2.8 MB/s eta 0:00:04
   ------ --------------------------------- 1.7/11.0 MB 3.4 MB/s eta 0:00:03
   -------- ------------------------------- 2.5/11.0 MB 4.6 MB/s eta 0:00:02
  

In [ ]:
ds = pd.read_csv('all-material-excel.csv', header=None)
ds = ds.sample(frac=1).reset_index(drop=True)
X1_train = np.array(ds[[0]])
X2_train = np.array(ds[[1]])
X3_train = np.array(ds[[2]])
X4_train = np.array(ds[[3]])
X5_train = np.array(ds[[4]])

y_train = np.array(ds.iloc[:, 5:])



In [ ]:
minX3 = np.min(X3_train)
maxX3 = np.max(X3_train)
minX4 = np.min(X4_train)
maxX4 = np.max(X4_train)
minX5 = np.min(X5_train)
maxX5 = np.max(X5_train)

X3_train = (X3_train - minX3) / (maxX3 - minX3)
X4_train = (X4_train - minX4) / (maxX4 - minX4)
X5_train = (X5_train - minX5) / (maxX5 - minX5)

X_train = np.column_stack((X1_train, X2_train, X3_train, X4_train, X5_train))


In [ ]:
x_train,x_test,Y_train,Y_test=train_test_split(X_train,y_train,test_size=0.1)
x_train,x_val,Y_train,y_val=train_test_split(x_train,Y_train,test_size=0.1)

In [ ]:
def objective(trial):
    num_layers=trial.suggest_int('num_layers_m' , 1, 10)
    num_neurons=trial.suggest_int('num_neurons' ,100, 2000)
    lr=trial.suggest_float('lr', 0.0001, 0.001, log=True)

    model=Sequential()
    model.add(InputLayer(input_shape=X_train[0].shape))
    for i in range(num_layers):
        model.add(Dense(num_neurons, activation='relu'))
        model.add(Dropout(0.2))

    model.add(Dense(200, activation='linear'))
    optimizer=tf.keras.optimizers.Adam(lr)
    model.compile(optimizer=optimizer,
                  loss='mse',
                  metrics='mse')
    early=EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)
    model.fit(x_train, Y_train, validation_data=(x_val, y_val), batch_size=128, epochs=500, verbose=0, shuffle=True, callbacks=[ early])
    score=model.evaluate(x_test, Y_test)
    return score[0]


In [ ]:
study= optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial= study.best_trial
print(" Value: {}".format(trial.value))
print("  Params  ")
for key, value in trial.params.items():
    print("  {}: {}".format(key, value))